In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [2]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('G:/Mi unidad/Maestria Data Mining/2023_DMEyF/datasets/competencia_01_crudo.csv')

,Success


In [3]:
%%sql
DROP TABLE IF EXISTS temp_table

,Success


In [4]:
%%sql
CREATE TEMPORARY TABLE temp_table AS
SELECT *,
  CASE
    WHEN EXISTS (
      SELECT True
      FROM competencia_01 c3
      WHERE c3.numero_de_cliente = c1.numero_de_cliente
        AND c3.foto_mes = c1.foto_mes+2) 
    THEN 'CONTINUA'
    WHEN (c1.foto_mes + 1) > (SELECT MAX(foto_mes) FROM competencia_01) THEN 'NA'
    WHEN NOT EXISTS (
      SELECT True
      FROM competencia_01 c2
      WHERE c2.numero_de_cliente = c1.numero_de_cliente
        AND c2.foto_mes = c1.foto_mes+1) 
    THEN 'BAJA+1'
    WHEN (c1.foto_mes + 2) > (SELECT MAX(foto_mes) FROM competencia_01) THEN 'NA'
    WHEN NOT EXISTS (
      SELECT True
      FROM competencia_01 c3
      WHERE c3.numero_de_cliente = c1.numero_de_cliente
        AND c3.foto_mes = c1.foto_mes+2) 
    THEN 'BAJA+2'
    ELSE 'NA'
      END AS clase_ternaria
FROM competencia_01 c1


,Success


In [5]:
%%sql
SELECT 
  clase_ternaria,
  foto_mes,
  COUNT(*) AS cantidad
FROM temp_table
GROUP BY clase_ternaria,foto_mes
ORDER BY foto_mes;

,clase_ternaria,foto_mes,cantidad
0,CONTINUA,202103,161312
1,BAJA+2,202103,963
2,BAJA+1,202103,1049
3,NA,202104,162668
4,BAJA+1,202104,969
5,NA,202105,164102


In [7]:
%%sql
COPY temp_table TO 'G:/Mi unidad/Maestria Data Mining/2023_DMEyF/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


In [17]:
%%sql
DROP TABLE IF EXISTS temp_table_2

,Success


In [6]:
%%sql
CREATE TEMPORARY TABLE temp_table_2 AS
SELECT
    *,
    ifnull(ctarjeta_visa_transacciones, 0) + ifnull(ctarjeta_master_transacciones, 0) as tc_transacciones_total,
    ifnull(ctarjeta_visa, 0) + ifnull(ctarjeta_master, 0) as tc_ctarjeta_total,
    ifnull(mtarjeta_visa_consumo, 0) + ifnull(mtarjeta_master_consumo, 0) as tc_consumo_total,
    ifnull(Master_mconsumototal, 0) + ifnull(Visa_mconsumototal, 0) as tc_mconsumo_total,
    ifnull(Visa_mlimitecompra, 0) + ifnull(Master_mlimitecompra, 0) as tc_limcompra_total,
    ifnull(Master_mpagado, 0) + ifnull(Visa_mpagado, 0) as tc_pagado_total,
    ifnull(Master_mpagominimo, 0) + ifnull(Visa_mpagominimo, 0) as tc_pagomin_total,
    ifnull(mprestamos_personales, 0) + ifnull(mprestamos_prendarios, 0) + ifnull(mprestamos_hipotecarios, 0) as pasivos_total,
    ifnull(mplazo_fijo_dolares, 0) + ifnull(mplazo_fijo_pesos, 0) + ifnull(minversion1_pesos, 0) + 
        ifnull(minversion1_dolares, 0) + ifnull(minversion2, 0) as activos_total,
    ifnull(mpayroll, 0) + ifnull(mpayroll2, 0) as flujos_ingresos_total,
    ifnull(mcajeros_propios_descuentos, 0) + ifnull(mtarjeta_visa_descuentos, 0) + ifnull(mtarjeta_master_descuentos, 0) as beneficios_total,
    ifnull(mcomisiones_mantenimiento, 0) + ifnull(mcomisiones_otras, 0) as comisiones_total,
    ifnull(mcuenta_debitos_automaticos, 0) + ifnull(mttarjeta_visa_debitos_automaticos, 0) + ifnull(mttarjeta_master_debitos_automaticos, 0) 
        + ifnull(mpagodeservicios, 0) + ifnull(mpagomiscuentas, 0) as gastos_total,
    
    NULLIF(tc_consumo_total / NULLIF(flujos_ingresos_total,0),0) as consumo_sobre_ingresos,
    NULLIF(tc_mconsumo_total / NULLIF(flujos_ingresos_total,0),0) as tcm_consumo_sobre_ingresos,
    NULLIF(tc_pagado_total / NULLIF(flujos_ingresos_total,0),0) as pagado_sobre_ingresos,
    NULLIF(tc_pagado_total / NULLIF(activos_total,0),0) as pagado_sobre_activos,
    NULLIF(tc_pagomin_total / NULLIF(flujos_ingresos_total,0),0) as pagomin_sobre_ingresos,
    NULLIF(tc_consumo_total / NULLIF(activos_total,0),0) as gastos_sobre_activos,
    NULLIF(activos_total / NULLIF(pasivos_total,0),0) as activos_sobre_pasivos,
    NULLIF(beneficios_total / NULLIF(comisiones_total,0),0) as beneficios_sobre_comisiones,
    NULLIF(beneficios_total / NULLIF(tc_consumo_total,0),0) as beneficios_sobre_consumos,
    NULLIF(gastos_total / NULLIF(flujos_ingresos_total,0),0) as gastos_sobre_ingresos,
    NULLIF(tc_limcompra_total / NULLIF(tc_consumo_total,0),0) as lim_sobre_consumos,
       
FROM temp_table

,Success


In [15]:
%%sql
ALTER TABLE temp_table_2 DROP COLUMN ctarjeta_visa_transacciones;
ALTER TABLE temp_table_2 DROP COLUMN ctarjeta_master_transacciones;
ALTER TABLE temp_table_2 DROP COLUMN ctarjeta_visa;
ALTER TABLE temp_table_2 DROP COLUMN ctarjeta_master;
ALTER TABLE temp_table_2 DROP COLUMN mtarjeta_visa_consumo;
ALTER TABLE temp_table_2 DROP COLUMN mtarjeta_master_consumo;
ALTER TABLE temp_table_2 DROP COLUMN Master_mconsumototal;
ALTER TABLE temp_table_2 DROP COLUMN Visa_mconsumototal;
ALTER TABLE temp_table_2 DROP COLUMN Visa_mlimitecompra;
ALTER TABLE temp_table_2 DROP COLUMN Master_mlimitecompra;
ALTER TABLE temp_table_2 DROP COLUMN Master_mpagado;
ALTER TABLE temp_table_2 DROP COLUMN Visa_mpagado;
ALTER TABLE temp_table_2 DROP COLUMN Master_mpagominimo;
ALTER TABLE temp_table_2 DROP COLUMN Visa_mpagominimo;


,Success


In [18]:
%%sql
CREATE TEMPORARY TABLE temp_table_3 AS
SELECT 
    *,
    cume_dist() over (partition by foto_mes order by tc_transacciones_total) as tc_transacciones_total_sd,
    cume_dist() over (partition by foto_mes order by tc_mconsumo_total) as tc_mconsumo_total_sd,
    cume_dist() over (partition by foto_mes order by tc_limcompra_total) as tc_limcompra_total_sd,
    cume_dist() over (partition by foto_mes order by tc_pagado_total) as tc_pagado_total_sd,
    cume_dist() over (partition by foto_mes order by pasivos_total) as pasivos_total_sd,
    cume_dist() over (partition by foto_mes order by activos_total) as activos_total_sd,
    cume_dist() over (partition by foto_mes order by flujos_ingresos_total) as flujos_ingresos_total_sd,
    cume_dist() over (partition by foto_mes order by comisiones_total) as comisiones_total_sd,
    cume_dist() over (partition by foto_mes order by beneficios_total) as beneficios_total_sd,
    cume_dist() over (partition by foto_mes order by gastos_total) as gastos_total_sd,
    cume_dist() over (partition by foto_mes order by tc_pagomin_total) as tc_pagomin_total_sd

FROM temp_table_2

,Success


In [19]:
%%sql
ALTER TABLE temp_table_3 DROP COLUMN tc_transacciones_total;
ALTER TABLE temp_table_3 DROP COLUMN tc_mconsumo_total;
ALTER TABLE temp_table_3 DROP COLUMN tc_limcompra_total;
ALTER TABLE temp_table_3 DROP COLUMN tc_pagado_total;
ALTER TABLE temp_table_3 DROP COLUMN pasivos_total;
ALTER TABLE temp_table_3 DROP COLUMN activos_total;
ALTER TABLE temp_table_3 DROP COLUMN flujos_ingresos_total;
ALTER TABLE temp_table_3 DROP COLUMN comisiones_total;
ALTER TABLE temp_table_3 DROP COLUMN beneficios_total;
ALTER TABLE temp_table_3 DROP COLUMN gastos_total;
ALTER TABLE temp_table_3 DROP COLUMN tc_pagomin_total;

,Success


In [20]:
%%sql
COPY temp_table_3 TO 'G:/Mi unidad/Maestria Data Mining/2023_DMEyF/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

,Success
